# Assignment 1

## Important notes
**Submission deadline:**
* **Thursday, 12.03.2020**

**Points: 13 + 2bp**

This assignment is meant to test your skills in course pre-requisites:  Scientific Python programming and  Machine Learning. If it is hard, I strongly advise you to drop the course.

Please use GitHub’s [pull requests](https://help.github.com/en/github/collaborating-with-issues-and-pull-requests/about-pull-requests) and issues to send corrections!

You can solve the assignment in any system you like, but we encourage you to try out [Google Colab](https://colab.research.google.com/).

## Assignment text
1. **[1p]** Download data competition from a Kaggle competition on sentiment prediction from [[https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data)].  Keep only full sentences, i.e. for each `SenteceId` keep only the entry with the lowest `PhraseId`.  Use first 7000 sentences as a `train set` and the remaining 1529 sentences as the `test set`. 

2. **[1p]** Prepare the data for logistic regression:
	Map the sentiment scores $0,1,2,3,4$ to a probability of the sentence being by setting $p(\textrm{positive}) = \textrm{sentiment}/4$.
	Build a dictionary of at most 20000 most frequent words.

3. **[3p]** Treat each document as a bag of words. e.g. if the vocabulary is 
	```
	0: the
	1: good
	2: movie
	3: is
	4: not
	5: a
	6: funny
	```
	Then the encodings can be:
	```
	good:                           [0,1,0,0,0,0,0]
	not good:                       [0,1,0,0,1,0,0] 
	the movie is not a funny movie: [1,0,2,1,1,1,1]
	```
    Train a logistic regression model to predict the sentiment. Compute the correlation between the predicted probabilities and the sentiment. Record the most positive and negative words.
    Please note that in this model each word gets its sentiment parameter $S_w$ and the score for a sentence is 
    $$\text{score}(\text{sentence}) = \sum_{w\text{ in sentence}}S_w$$

4. **[3p]** Now prepare an encoding in which negation flips the sign of the following words. For instance for our vocabulary the encodings become:
	```
	good:                           [0,1,0,0,0,0,0]
	not good:                       [0,-1,0,0,1,0,0]
	not not good:                   [0,1,0,0,0,0,0]
	the movie is not a funny movie: [1,0,0,1,1,-1,-1]
	```
	For best results, you will probably need to construct a list of negative words.
	
	Again train a logistic regression classifier and compare the results to the Bag of Words approach.
	
	Please note that this model still maintains a single parameter for each word, but now the sentence score is
	$$\text{score}(\text{sentence}) = \sum_{w\text{ in sentence}}-1^{\text{count of negations preceeding }w}S_w$$

5. **[5p]** Now also consider emphasizing words such as `very`. They can boost (multiply by a constant >1) the following words.
	Implement learning the modifying multiplier for negation and for emphasis. One way to do this is to introduce a model which has:
	- two modifiers, $N$ for negation and $E$ for emphasis
	- a sentiment score $S_w$ for each word 
And score each sentence as:
$$\text{score}(\text{sentence}) = \sum_{w\text{ in sentence}}N^{\text{\#negs prec. }w}E^{\text{\#emphs prec. }w}S_w$$

You will need to implement a custom logistic regression model to support it.

6. **[2pb]** Propose, implement, and evaluate an extension to the above model.


## 1.

Kaggle data

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

# Install Kaggle library
!pip install -q kaggle
#!cp ~/kaggle.json ~/.kaggle/kaggle.json

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Upload kaggle API key file
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [0]:
!ls

 gdrive		    kaggle.json   sampleSubmission.csv	 train.tsv.zip
'kaggle (1).json'   sample_data   test.tsv.zip


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [0]:
!kaggle competitions download -c sentiment-analysis-on-movie-reviews

train.tsv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.tsv.zip: Skipping, found more recently modified local copy (use --force to force download)
sampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!ls

 gdrive		    kaggle.json   sampleSubmission.csv	 train.tsv.zip
'kaggle (1).json'   sample_data   test.tsv.zip


In [0]:
import pandas as pd

# Import the test and train datasets into pandas dataframe
df_train = pd.read_csv('train.tsv.zip', header = 0, compression = 'zip', sep = '\t', quotechar = '"')
df_test = pd.read_csv('test.tsv.zip', header = 0, compression = 'zip', sep = '\t', quotechar = '"')
df_sample = pd.read_csv('sampleSubmission.csv', header = 0, sep = ',', quotechar = '"')

In [0]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


Only full sentences - the entries with the lowest *PhraseId*.

In [0]:
df_train_full = df_train.loc[df_train.groupby(by='SentenceId')['PhraseId'].idxmin()].reset_index(drop=True)
df_train_full

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,64,2,"This quiet , introspective and entertaining in...",4
2,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
3,117,4,A positively thrilling combination of ethnogra...,3
4,157,5,Aggressive self-glorification and a manipulati...,1
...,...,...,...,...
8524,155985,8540,... either you 're willing to go with this cla...,2
8525,155998,8541,"Despite these annoyances , the capable Claybur...",2
8526,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1
8527,156032,8543,The movie 's downfall is to substitute plot fo...,1


**Train set** - first 7000 sentences, **test set** - remaining 1529.

In [0]:
train_full = df_train_full.head(7000)
test_full = df_train_full.tail(1529)

## 2.

Preparing the data for logistic regression.    
Probability: $p(\textrm{positive}) = \textrm{sentiment}/4$

In [0]:
train_full.assign(Probability = train_full['Sentiment']/4)

,PhraseId,SentenceId,Phrase,Sentiment,Probability
0,1,1,A series of escapades demonstrating the adage ...,1,0.25
1,64,2,"This quiet , introspective and entertaining in...",4,1.00
2,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1,0.25
3,117,4,A positively thrilling combination of ethnogra...,3,0.75
4,157,5,Aggressive self-glorification and a manipulati...,1,0.25
...,...,...,...,...,...
6995,130088,7007,"Snoots will no doubt rally to its cause , trot...",1,0.25
6996,130131,7008,It 's better suited for the history or biograp...,1,0.25
6997,130161,7009,Buries an interesting storyline,2,0.50
6998,130181,7010,This one is a few bits funnier than Malle 's d...,3,0.75


20000 most frequent words

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [0]:
vec = CountVectorizer()
bag_of_words = vec.fit_transform(train_full['Phrase'])
sum_words = bag_of_words.sum(axis = 0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
most_freq = dict(words_freq[:20000])
len(words_freq)

13796

There are only 13796 words, so I'm going to show 50 most common words.

In [0]:
#most_freq

In [0]:
pd.DataFrame.from_dict(most_freq, orient = 'index')[:50]

,0
the,5942
and,3684
of,3605
to,2495
is,2088
it,1996
that,1573
in,1559
as,1044
but,960


## 3.
$$\text{score}(\text{sentence}) = \sum_{w\text{ in sentence}}S_w$$

Bag of words

In [0]:
bag_of_words.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
import numpy as np

In [0]:
np.unique(bag_of_words.toarray())

array([0, 1, 2, 3, 4, 5, 6, 7])

Training a logistic regression model.

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

In [0]:
text_clf = Pipeline([('vect', CountVectorizer()), ('clf', LogisticRegression())])
text_clf.fit(train_full['Phrase'], train_full['Sentiment'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                          

In [0]:
predicted = text_clf.predict(test_full['Phrase'])
np.mean(predicted == test_full['Sentiment'])

0.39045127534336166

In [0]:
cm1 = confusion_matrix(test_full['Sentiment'], predicted)

Correlation between the predicted probabilities and the sentiment - I think I didn't understand the task, so I did something but I'm not sure if it makes sense.

In [0]:
lr = LogisticRegression()
lr = lr.fit(bag_of_words, train_full['Sentiment'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
np.corrcoef(lr.predict_proba(bag_of_words), [0,1,2,3,4])

array([[ 1.        , -0.32714686,  0.98207972, ..., -0.10528025,
        -0.33783629, -0.34560485],
       [-0.32714686,  1.        , -0.2395563 , ..., -0.21517063,
         0.06252968,  0.78522535],
       [ 0.98207972, -0.2395563 ,  1.        , ..., -0.19301655,
        -0.39813793, -0.37080915],
       ...,
       [-0.10528025, -0.21517063, -0.19301655, ...,  1.        ,
         0.94975275,  0.29495314],
       [-0.33783629,  0.06252968, -0.39813793, ...,  0.94975275,
         1.        ,  0.50909045],
       [-0.34560485,  0.78522535, -0.37080915, ...,  0.29495314,
         0.50909045,  1.        ]])

The most positive and negative words

In [0]:
# indices of the sorted cofficients
coeff = lr.coef_
inds_neg = np.argsort(coeff[[0]])
inds_pos = np.argsort(coeff[[4]])
feature_names = np.array(vec.get_feature_names())

In [0]:
# negative (highest coeffs for sentiment = 0)
for i in range(25):
    print(feature_names[inds_neg[0,-i]])

best
worst
stupid
devoid
dull
terrible
disappointment
horrible
unpleasant
mess
bad
incoherent
insulting
crap
unbearably
ugly
awful
purpose
fails
poor
lazy
generic
horribly
morbid
snooze


In [0]:
# positive (highest coeffs for sentiment = 4)
for i in range(25):
    print(feature_names[inds_pos[0,-i]])

less
masterpiece
beautifully
best
captivating
remarkable
entertaining
moving
years
dazzling
riveting
beautiful
powerful
wonderful
fun
brilliant
fantastic
thoughtful
amazing
rare
masterful
enjoyable
impressive
imax
delights


## 4.
Encoding in which negation flips the sign of the following words.

$$\text{score}(\text{sentence}) = \sum_{w\text{ in sentence}}-1^{\text{count of negations preceeding }w}S_w$$


In [0]:
def df(text, col_names):
    return pd.DataFrame(np.zeros((len(text), len(col_names))), columns = col_names)

In [0]:
from string import punctuation

def my_tokenizer(text):
    text_lst = text.split()
    for word in text_lst:
        text_lst[text_lst.index(word)] = word.lower().strip(punctuation)
    for word in text_lst:
        if word is '':
            text_lst.remove(word)
    return text_lst

In [0]:
full_lst = []
for sentence in train_full['Phrase'].values:
    full_lst.append(my_tokenizer(sentence))

lst_scores = []
for x in full_lst:
    lst_scores.append([1]*len(x))


In [0]:
one_lst = sum(full_lst, [])
one_lst = list(set(one_lst))
one_lst.sort()
one_lst

['',
 '1',
 '1.2',
 '1.8',
 '10',
 '10-course',
 '10-year-old',
 '100',
 '100-minute',
 '100-year',
 '101',
 '102-minute',
 '103-minute',
 '104',
 '105',
 '10th',
 '10th-grade',
 '11',
 '110',
 '112-minute',
 '12',
 '12-year-old',
 '120',
 '127',
 '129-minute',
 '12th',
 '13',
 '13th',
 '14-year-old',
 '140',
 '146',
 '15',
 '15th',
 '163',
 '168-minute',
 '170',
 '1790',
 '18',
 '18-year-old',
 '1899',
 '1915',
 '1920',
 '1930s',
 '1933',
 '1937',
 '1938',
 '1940s',
 '1950',
 '1950s',
 '1952',
 '1953',
 '1957',
 '1958',
 '1959',
 '1960s',
 '1962',
 '1970',
 '1970s',
 '1971',
 '1972',
 '1973',
 '1975',
 '1980',
 '1980s',
 '1984',
 '1986',
 '1989',
 '1990',
 '1992',
 '1994',
 '1995',
 '1997',
 '1999',
 '19th',
 '19th-century',
 '1\\/2',
 '2',
 '20',
 '2000',
 '2001',
 '2002',
 '20th',
 '20th-century',
 '21\\/2',
 '21st',
 '22',
 '22-year-old',
 '24-and-unders',
 '2455',
 '25',
 '26',
 '26-year-old',
 '270',
 '295',
 '2\\/3',
 '3',
 '3-d',
 '3-year-olds',
 '30',
 '30-year',
 '300',
 '300

**Everything in the function:**

In [0]:
def neg_vectorizer(txt, data, col_names):
    negations = ['not', 'no', 'none', 'never', 'neither', 'angry', 'annoy', 'anxious', 'annoying'
              'awful', 'aggresive', 'afraid', 'bad', 'banal', 'awfully',
              'boring', 'confused', 'crap', 'crazy', 'creepy', 'cruel', 
              'cry', 'damage', 'damaging', 'confusing', 'dead', 'depressed',
              'dirty', 'disease', 'disgusting', 'disappointing', 'disappointment',
              'difficult', 'disaster', 'disastrous', 'dangerous', 'dull', 'excuse',
              'fail', 'fatal', 'fear', 'frighten', 'frightful', 'freaky', 'failed',
              'grimace', 'grotesque', 'guilty', 'hard', 'harmful', 'hate', 'horrible',
              'hurt', 'hurtful', 'horribly', 'icky', 'ignore', 'immature', 'imperfect', 'impossible', 
              'inelegant', 'insane', 'junky', 'lose', 'lousy', 'lumpy', 'lazy', 'little', 'zero', 'yell', 
              'worthless', 'ugly', 'unfair', 'unfavorable', 'unhappy', 'unlucky', 
              'unpleasant', 'unsatisfactory', 'unwanted','unwholesome', 
              'unwise', 'upset', 'unsuitable', 'unproffesional', 'uninteresting', 'terrible', 'terrifying',
              'threatening', 'trouble', 'tedious', 'sad', 'scare', 'scary', 'scream', 'shocking', 'sick', 'sickening', 'sorry', 
              'stressful', 'stuck', 'stupid', 'silly', 'reject', 'rude', 'quit',
              'questionable', 'mean', 'messy', 'missing', 'misunderstood',
              'mistake', 'manipulation', 'mess', 'naive', 'negate', 'negative', 'nobody', 
              'nonsense', 'old', 'overrated', 'pain', 'pessimistic', 'poor', 'pumped']

    full_lst = []
    for sentence in txt:
        full_lst.append(my_tokenizer(sentence))

    lst_scores = []
    for x in full_lst:
        lst_scores.append([1]*len(x)) # na początek, każdemu wyrazowi w zdaniu przypisuję automatycznie 1 (=(-1)^0)

    j = 0
    for lst in full_lst:
        l = len(lst)
        for i in range(len(lst)-1):
            if lst[i] in negations: # po negatywnym wyrazie zmieniamy znak 'score' wyrazów na prawo
                lst_scores[j][i+1:] = [-1*x for x in lst_scores[j][i+1:]]
        for k in range(len(lst)):
            try:
                # dodajemy scores do danego wiersza w tabeli wszystkich wyrazów
                data.iloc[j][col_names.index(lst[k])] += lst_scores[j][k]
            except ValueError:
                pass
        j += 1

    return data

In [0]:
train_data_2 = neg_vectorizer(train_full['Phrase'].values, df(train_full['Phrase'], one_lst), one_lst)

In [0]:
np.unique(train_data_2.values)

array([-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.])

In [0]:
test_data_2 = neg_vectorizer(test_full['Phrase'].values, df(test_full['Phrase'], one_lst), one_lst)

Logistic regression

In [0]:
lr2 = LogisticRegression()
lr2.fit(train_data_2.values, train_full['Sentiment'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predicted2 = lr2.predict(test_data_2.values)
np.mean(predicted2 == test_full['Sentiment'])

0.3433616742969261

In [0]:
cm2 = confusion_matrix(test_full['Sentiment'], predicted2)

In [0]:
cm1 # previous model

array([[ 46,  86,  26,  27,   9],
       [ 58, 177,  79,  57,  19],
       [ 23,  84,  84,  85,  17],
       [ 10,  67,  72, 200,  70],
       [  4,  16,  27,  96,  90]])

In [0]:
cm2 # model with negations

array([[ 37,  76,  33,  38,  10],
       [ 49, 170,  76,  78,  17],
       [ 23, 100,  63,  86,  21],
       [ 19,  73,  74, 179,  74],
       [  2,  20,  31, 104,  76]])

It doesn't work better than CountVectorizer. The reason for this may be text preparation - I just used a simple tokenization. 

## 5.
$$\text{score}(\text{sentence}) = \sum_{w\text{ in sentence}}N^{\text{\#negs prec. }w}E^{\text{\#emphs prec. }w}S_w$$

In [0]:
def neg_emphasize_vectorizer(txt, data, col_names, N, E):
    negations = ['not', 'no', 'none', 'never', 'neither', 'angry', 'annoy', 'anxious', 'annoying'
              'awful', 'aggresive', 'afraid', 'bad', 'banal', 'awfully',
              'boring', 'confused', 'crap', 'crazy', 'creepy', 'cruel', 
              'cry', 'damage', 'damaging', 'confusing', 'dead', 'depressed',
              'dirty', 'disease', 'disgusting', 'disappointing', 'disappointment',
              'difficult', 'disaster', 'disastrous', 'dangerous', 'dull', 'excuse',
              'fail', 'fatal', 'fear', 'frighten', 'frightful', 'freaky', 'failed',
              'grimace', 'grotesque', 'guilty', 'hard', 'harmful', 'hate', 'horrible',
              'hurt', 'hurtful', 'horribly', 'icky', 'ignore', 'immature', 'imperfect', 'impossible', 
              'inelegant', 'insane', 'junky', 'lose', 'lousy', 'lumpy', 'lazy', 'little', 'zero', 'yell', 
              'worthless', 'ugly', 'unfair', 'unfavorable', 'unhappy', 'unlucky', 
              'unpleasant', 'unsatisfactory', 'unwanted','unwholesome', 
              'unwise', 'upset', 'unsuitable', 'unproffesional', 'uninteresting', 'terrible', 'terrifying',
              'threatening', 'trouble', 'tedious', 'sad', 'scare', 'scary', 'scream', 'shocking', 'sick', 'sickening', 'sorry', 
              'stressful', 'stuck', 'stupid', 'silly', 'reject', 'rude', 'quit',
              'questionable', 'mean', 'messy', 'missing', 'misunderstood',
              'mistake', 'manipulation', 'mess', 'naive', 'negate', 'negative', 'nobody', 
              'nonsense', 'old', 'overrated', 'pain', 'pessimistic', 'poor', 'pumped']
    
    emphasize = ['very', 'strongly', 'really', 'seriously', 'wonderful']

    full_lst = []
    for sentence in txt:
        full_lst.append(my_tokenizer(sentence))

    lst_scores = []
    for x in full_lst:
        lst_scores.append([1]*len(x)) # na początek, każdemu wyrazowi w zdaniu przypisuję automatycznie 1 (=(-1)^0)

    j = 0
    for lst in full_lst:
        l = len(lst)
        for i in range(len(lst)-1):
            if lst[i] in emphasize: # po negatywnym wyrazie mnożymy wyrazy na prawo przez N, po uwydatnieniu przez E
                lst_scores[j][i+1:] = [E*x for x in lst_scores[j][i+1:]]
            elif lst[i] in negations:
                lst_scores[j][i+1:] = [N*x for x in lst_scores[j][i+1:]]
        for k in range(len(lst)):
            try:
                # dodajemy scores do danego wiersza w tabeli wszystkich wyrazów
                data.iloc[j][col_names.index(lst[k])] += lst_scores[j][k]
            except ValueError:
                pass
        j += 1

    return data

##### N = -1, E = 3

In [0]:
train_data_31 = neg_emphasize_vectorizer(train_full['Phrase'].values, df(train_full['Phrase'], one_lst), one_lst, -1, 3)

In [0]:
np.unique(train_data_31)

array([-27., -18., -10.,  -9.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,
         1.,   2.,   3.,   4.,   5.,   6.,   7.,   9.,  10.,  13.,  15.,
        27.,  28.,  31.,  56.])

In [0]:
test_data_31 = neg_emphasize_vectorizer(test_full['Phrase'].values, df(test_full['Phrase'], one_lst), one_lst, -1, 3)

In [0]:
lr31 = LogisticRegression()
lr31.fit(train_data_31.values, train_full['Sentiment'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predicted31 = lr31.predict(test_data_31.values)
np.mean(predicted31 == test_full['Sentiment'])

0.3433616742969261

In [0]:
confusion_matrix(test_full['Sentiment'], predicted31)

array([[ 36,  73,  33,  42,  10],
       [ 52, 169,  73,  78,  18],
       [ 24,  98,  65,  86,  20],
       [ 22,  73,  71, 184,  69],
       [  4,  23,  30, 105,  71]])

##### N = -1, E = 5

In [0]:
train_data_32 = neg_emphasize_vectorizer(train_full['Phrase'].values, df(train_full['Phrase'], one_lst), one_lst, -1, 5)

In [0]:
np.unique(train_data_32)

array([-125.,  -50.,  -26.,  -25.,  -20.,  -15.,  -10.,   -7.,   -6.,
         -5.,   -4.,   -3.,   -2.,   -1.,    0.,    1.,    2.,    3.,
          4.,    5.,    6.,    7.,    8.,   10.,   11.,   13.,   15.,
         16.,   21.,   25.,   26.,   31.,  125.,  126.,  131.,  252.])

In [0]:
test_data_32 = neg_emphasize_vectorizer(test_full['Phrase'].values, df(test_full['Phrase'], one_lst), one_lst, -1, 5)

In [0]:
lr32 = LogisticRegression()
lr32.fit(train_data_32.values, train_full['Sentiment'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predicted32 = lr32.predict(test_data_32.values)
np.mean(predicted32 == test_full['Sentiment'])

0.34793982995421846

In [0]:
confusion_matrix(test_full['Sentiment'], predicted32)

array([[ 35,  78,  31,  40,  10],
       [ 50, 171,  75,  78,  16],
       [ 25,  96,  67,  83,  22],
       [ 23,  71,  71, 184,  70],
       [  4,  24,  32,  98,  75]])

We have slightly better results in prediction of positive phrases (but still it's not a big difference). To change predictions for negative phrases, I'll try to change multiplier for negative words.

##### N = -3, E = 5

In [0]:
train_data_33 = neg_emphasize_vectorizer(train_full['Phrase'].values, df(train_full['Phrase'], one_lst), one_lst, -3, 5)

In [0]:
np.unique(train_data_33)

array([-375., -150.,  -81.,  -78.,  -75.,  -70.,  -54.,  -53.,  -45.,
        -32.,  -30.,  -27.,  -26.,  -24.,  -21.,  -20.,  -18.,  -15.,
        -14.,  -13.,  -12.,  -11.,  -10.,   -9.,   -8.,   -6.,   -5.,
         -4.,   -3.,   -2.,   -1.,    0.,    1.,    2.,    3.,    4.,
          5.,    6.,    7.,    8.,    9.,   10.,   11.,   12.,   13.,
         15.,   16.,   18.,   19.,   24.,   25.,   26.,   27.,   30.,
         31.,   36.,   39.,   42.,   45.,   46.,   50.,   61.,   81.,
         90.,  125.,  126.,  131.,  135.,  225.,  252.])

In [0]:
test_data_33 = neg_emphasize_vectorizer(test_full['Phrase'].values, df(test_full['Phrase'], one_lst), one_lst, -3, 5)

In [0]:
lr33 = LogisticRegression()
lr33.fit(train_data_33.values, train_full['Sentiment'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predicted33 = lr33.predict(test_data_33.values)
np.mean(predicted33 == test_full['Sentiment'])

0.3198168737737083

In [0]:
confusion_matrix(test_full['Sentiment'], predicted33)

array([[ 27,  74,  32,  42,  19],
       [ 45, 144,  81,  98,  22],
       [ 21,  84,  67,  89,  32],
       [ 20,  75,  74, 176,  74],
       [  7,  24,  33,  94,  75]])

All in all, emphasizing the words improves the model, but changing the multiplier for negations don't.